In [ ]:
!git clone https://github.com/LC1332/Luotuo-QA.git ./luotuo_qa
!cd ./luotuo_qa && pip install -r requirements.txt

In [ ]:
from luotuo_qa.app.infer import get_model, infer

from transformers import AutoTokenizer

model_name: str = "THUDM/chatglm-6b"
peft_path: str = "silk-road/luotuo-qa-lora-0.1"
model_revision: str = "969290547e761b20fdb96b0602b4fd8d863bbb85"
with_origin_model: bool = True

model = get_model(model_name, peft_path)
origin_model = None
if with_origin_model:
    origin_model = get_model(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, revision = model_revision)

In [ ]:
example_story = '''
长妈妈曾经讲给我一个故事听：先前，有一个读书人住在古庙里用功，晚间， 在院子里纳凉的时候，突然听到有人在叫他。答应着，四面看时，却见一个美女的 脸露在墙头上，向他一笑，隐去了。他很高兴；但竟给那走来夜谈的老和尚识破了 机关。说他脸上有些妖气，一定遇见“美女蛇”了；这是人首蛇身的怪物，能唤人 名，倘一答应，夜间便要来吃这人的肉的。他自然吓得要死，而那老和尚却道无妨 ，给他一个小盒子，说只要放在枕边，便可高枕而卧。他虽然照样办，却总是睡不 着，——当然睡不着的。到半夜，果然来了，沙沙沙！门外象是风雨声。他正抖作 一团时，却听得豁的一声，一道金光从枕边飞出，外面便什么声音也没有了，那金 光也就飞回来，敛在盒子里。后来呢？后来，老和尚说，这是飞蜈蚣，它能吸蛇的 脑髓，美女蛇就被它治死了。
'''
example_question = '谁遇到了美女蛇？'

import gradio as gr
def main():
    with gr.Blocks() as b:
        with gr.Row():
            with gr.Row():
                with gr.Box():
                # UI: Story input
                    with gr.Row():
                        story = gr.TextArea(value=example_story, label="Story", lines=10, interactive=True)
                    with gr.Row():
                        with gr.Column(scale = 6):
                            # UI: Question input
                            question = gr.TextArea(value=example_question, label="Question", lines=2, interactive=True)
                        with gr.Column(scale = 1):
                            # UI: Submit button
                            submit = gr.Button("Ask", label="Submit", interactive=True, variant="primary")
        with gr.Row():
            with gr.Box():
                with gr.Row():
                    with gr.Column():
                        # UI: origin model output
                        with gr.Row():
                            origin_answer = gr.Textbox(label=model_name+":", lines=2, interactive=False)
                    with gr.Column():
                        # UI: Lora output
                        with gr.Box():
                            with gr.Row():
                                question_as = gr.Textbox(label="Question escapes to:", lines=2, interactive=False)
                            with gr.Row():
                                answer = gr.Textbox(label="Answer", lines=2, interactive=False)
                    with gr.Column():
                        # UI: Lora output^2
                        with gr.Box():
                            with gr.Row():
                                question_as_v2 = gr.Textbox(label="Question^2 escapes to:", lines=2, interactive=False)
                            with gr.Row():
                                answer_v2 = gr.Textbox(label="Answer^2:", lines=2, interactive=False)
        def inner_infer(story, question):
            return infer(model, tokenizer, story, question, origin_model = origin_model)
        submit.click(
                fn=inner_infer,
                inputs=[
                    story,
                    question,
                ],
                outputs=[
                    origin_answer,
                    question_as,
                    answer,
                    question_as_v2,
                    answer_v2,
                ],
                show_progress=True,
            )
    b.launch(prevent_thread_lock=True, share=True)

if __name__ == "__main__":
    main()

    import time
    while True:
        time.sleep(0.5)